# Demo the tool that checks for the presence of bright stars in the Claws susceptibility region

## Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pysiaf.utils import rotations
import matplotlib.patches as patches
from claws_tool import apt_program, compute_line
from apt_sql import Sqlfile
import pysiaf
from matplotlib.path import Path
import os

import warnings
warnings.filterwarnings('ignore')

%matplotlib notebook

**WARNING**: LOCAL JWST PRD VERSION PRDOPSSOC-062 CANNOT BE CHECKED AGAINST ONLINE VERSION


## Common catalog defintions

In [6]:
catargsSIMBAD={'inner_rad':8.,'outer_rad':12.,'sourcecat':'SIMBAD',
                                       'band':'K','maxmag':4.,'simbad_timeout':200,'verbose':True}
catargs2MASS={'inner_rad':8.,'outer_rad':12.,'sourcecat':'2MASS','2MASS_filename':'/Users/mfix/Desktop/sandbox/jwst_rogue_path_tool/Data/TwoMassForMario_km5.csv',
                                       'band':'k_m','maxmag':4,'verbose':True}

## First use case: review a program for potential presence of claws at any orientation

### Create a program object (ingests and parses an APT sql file)

In [7]:
filename = '/Users/mfix/Desktop/sandbox/jwst_rogue_path_tool/APT_demo_file/Demo_apt.sql'
program = apt_program(filename)

### Define properties of the catalog query

### Select which observations to test (use None to test all the observations)

In [8]:
obsids = [2,3]

#### First ingest the observations and get their corresponding catalog

In [9]:
program.add_observations(obsids=obsids, catargs=catargs2MASS, smallregion=False)

Adding obsid: 2
Total number of NIRCAM exposures in this observation: 6
Total number of NIRCAM exposure specifications: 3
Total number of visits: 3
Adopted inner and outer radius 7.963386910783606 12.036613089216395


UnitTypeError: Angle instances require units equivalent to 'rad', but no unit was given.

#### Run the checks on the possible V3PA values

In [10]:
program.check_observations()

#### Plot the results

In [11]:
for ob in program.observations:
    ob.plot_observations_checks()

## Second use case: verify that an assigned PA works well for a yet unexecuted observation

### Create a program object (ingests and parses an APT sql file)

In [ ]:
savedirroot = '/Users/gennaro/Desktop/'
PID = '4445'
filename = '/Users/gennaro/Desktop/Claws_prediction_tool/APT_sql/PID_{}.sql'.format(PID)
program = apt_program(filename)


savedir = savedirroot+'PID_'+PID+'/'
if os.path.isdir(savedir) ==  False:
    os.mkdir(savedir)



### Select which observation(s) to check

In [13]:
obsids = [3]

### Ingest and check them

In [14]:
program.add_observations(obsids=obsids,catargs=catargs2MASS,smallregion=False)
program.check_observations()

Adding obsid: 3
Total number of NIRCAM exposures in this observation: 4
Total number of NIRCAM exposure specifications: 2
Total number of visits: 2
Adopted inner and outer radius 7.963473984763377 12.036526015236623


UnitTypeError: Angle instances require units equivalent to 'rad', but no unit was given.

### Planned V3PA

In [15]:
att = np.linspace(159,181,12)
print(att)

[159. 161. 163. 165. 167. 169. 171. 173. 175. 177. 179. 181.]


In [16]:
rds = []
for a in att:

    rd = program.observations[0].fixed_angle(a,RP_padding=0.,nrows=1,ncols=2,draw_allexp=False,savefilenames=[None,'/Users/gennaro/Desktop/PID_4445/PID_4445_{:6.2f}.png'.format(a)])
    rds.append(rd)

IndexError: list index out of range

In [ ]:
#for rd in rds:
#    print(rd['V3PA'],rd['totcts_B'])
#    display(rd['catalog'].loc[rd['BM_ALL_B'],['RAdeg','DECdeg','k_m','avg_intensity_B','j_m_fict_mag_B','h_m_fict_mag_B','k_m_fict_mag_B']])

In [ ]:
program.observations

## Third use case: check an executed program at its fixed V3PA 

Can be used to see:

    - whether we have stars in the susceptibility region for observations that showed claws
    - whether we DO NOT have stars in the susceptibility region for observations that DO NOT show claws



### Subcase A :rely on the observation having been reported as affected by claws (ie the obs is listed in Ben's "claw_summary.txt" table)

#### Read in the claws_summary file

This file contains a list of observations that showed claws, and their executed V3PA

In [ ]:
df_obs_claws = pd.read_csv('/Users/gennaro/Desktop/Claws_prediction_tool/Data/claw_summary.txt',skiprows=2,names=['PID','obs','detector','pa_v3','claws','notes'])
df_obs_claws.head(10)

##### Pick one of the cases

In [ ]:
PID, obsid = 2738, 3
BM = (df_obs_claws['PID'] == PID) & (df_obs_claws['obs'] == obsid)
df_obs_claws[BM]

#### Read in the corresponding SQL file

In [ ]:
filename = '/Users/gennaro/Desktop/Claws_prediction_tool/APT_sql/PID_{}.sql'.format(PID)

#### Create a program object

In [ ]:
program = apt_program(filename)

#### Add just the affected observation

In [ ]:
program.add_observations(obsids=[obsid],catargs=catargs2MASS,smallregion=False)

#### Check the observation

In [ ]:
program.check_observations()

#### Run some summary diagnostic and get plots and a dataframe with the results

In [ ]:
rd = program.observations[0].fixed_angle(df_obs_claws.loc[BM,'pa_v3'].values[0],RP_padding=0.,nrows=2,ncols=3)

In [ ]:
rd['catalog'].head()

In [ ]:
df_A = rd['catalog'].filter(regex=("IN_A_exp*"))

BM_ALL_A = True
for c in df_A.columns:
    BM_ALL_A &= df_A[c]

df_B = rd['catalog'].filter(regex=("IN_B_exp*"))

BM_ALL_B = True
for c in df_B.columns:
    BM_ALL_B &= df_B[c]



print('Ben table')
display(df_obs_claws.loc[BM,['detector','claws','notes']])
print(' ')

print('Mod A results')
display(rd['catalog'].loc[BM_ALL_A,['RAdeg','DECdeg','k_m']])
print(' ')

print('Mod B results')
display(rd['catalog'].loc[BM_ALL_B,['RAdeg','DECdeg','k_m']])
print(' ')


### Subcase B : more general, does not rely on the observation being listed in Ben's "claw_summary.txt" table

#### Setup PID and obs and v3PA

In [ ]:
PID, obsid, pa_v3 = 1181, 98, 133.3177706698362

#### Read in the corresponding SQL file, setup proram and observation

In [ ]:
filename = '/Users/gennaro/Desktop/Claws_prediction_tool/APT_sql/PID_{}.sql'.format(PID)
program = apt_program(filename)
program.add_observations(obsids=[obsid],catargs=catargs2MASS,smallregion=False)

#### Check and run diagnostic for fixed angle

In [ ]:
program.check_observations()
rd = program.observations[0].fixed_angle(pa_v3,RP_padding=0.,nrows=2,ncols=3)

## Run a detailed sweep in V3 PA on one obs to determne the DN/pix/ks in claws measured on the detector and compare them to the expected background

### Setup PID and obs and v3PA

In [ ]:
#PID, obsid, pa_v3 = 1181, 98, 133.3177706698362
PID, obsid, pa_v3 = 1187, 35, 94.13

### Read in the corresponding SQL file, setup proram and observation

In [ ]:
filename = '/Users/gennaro/Desktop/Claws_prediction_tool/APT_sql/PID_{}.sql'.format(PID)
program = apt_program(filename)
program.add_observations(obsids=[obsid],catargs=catargs2MASS,smallregion=False)

### Run the sweep

In [ ]:
tmS,tcS = program.observations[0].DN_report(np.arange(0.,360.,3),RP_padding=0.,draw_reports=True,
                                                      background_params=[{'threshold':0.1,'func':np.min},{'threshold':0.2,'func':np.mean}])
#,
#                                                     save_report_dir='/Users/gennaro/Desktop/testc/',save_figures_dir='/Users/gennaro/Desktop/testc/')
#tmA,tmB, tcA, tcB = program.observations[0].DN_report(np.arange(0.,360.,1),RP_padding=0.,draw_reports=True,savefilename=None,
#                                                      background_params=[{'threshold':0.1,'func':np.min}])